In [ ]:
import cv2
import numpy as np
import time
 
def get_background(cap, capture_duration=3):
    print("Capturing background... Please step out of the frame.")
    start_time = time.time()
    background = None
 
    while True:
        ret, frame = cap.read()
 
        if not ret or frame is None or frame.size == 0:
            print("Warning: Invalid frame received. Retrying...")
            continue
 
        frame = np.flip(frame, axis=1).copy()
        elapsed = time.time() - start_time
 
        try:
            cv2.putText(frame, f"Capturing background in {int(capture_duration - elapsed)+1}s",
                        (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            cv2.imshow("Invisibility Cloak - Background Setup", frame)
        except Exception as e:
            print("OpenCV error during frame rendering:", e)
            continue
 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Cancelled by user.")
            return None
 
        if elapsed > capture_duration:
            background = frame
            break
 
    cv2.destroyWindow("Invisibility Cloak - Background Setup")
    print("Background captured.")
    return background
 
 
def create_mask(frame, lower_color, upper_color):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    return mask
 
def apply_invisibility_effect(frame, background, mask):
    mask_inv = cv2.bitwise_not(mask)
 
    res1 = cv2.bitwise_and(background, background, mask=mask)
    res2 = cv2.bitwise_and(frame, frame, mask=mask_inv)
    final = cv2.addWeighted(res1, 1, res2, 1, 0)
    return final
 
def run_cloak(color_name='red', record=False):
    cap = cv2.VideoCapture(0)
 
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return
    background = get_background(cap)
    if background is None:
        print("Error: Background not captured.")
        cap.release()
        return
 
    color_ranges = {
        'grey': [
            (np.array([0, 0, 40]), np.array([180, 50, 200]))  # HSV range for grey
        ],
        'black': [
            (np.array([0, 0, 0]), np.array([180, 255, 50]))  # HSV range for black
        ],
        'blue': [
            (np.array([100, 150, 0]), np.array([140, 255, 255]))
        ],
        'green': [
            (np.array([40, 40, 40]), np.array([70, 255, 255]))
        ],
        'iceblue': [
        (np.array([85, 10, 200]), np.array([100, 80, 255]))  # HSV range for ice blue
    ]
    }
    if color_name == 'red_green':
        masks = color_ranges['red'] + color_ranges['green']
    elif color_name in color_ranges:
        masks = color_ranges[color_name]
    else:
        print(f"Error: Unsupported color '{color_name}'. Choose from red, green, blue, or red_green.")
        cap.release()
        return
 
 
    if record:
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('invisibility_output_red.avi', fourcc, 20.0, (frame_width, frame_height))
    else:
        out = None
 
    print("Starting invisibility effect... Press 'q' to quit.")
 
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None or frame.size == 0:
            print("Warning: Skipping invalid frame.")
            continue
 
        frame = np.flip(frame, axis=1).copy()
 
        try:
            cv2.putText(frame, f"Wear the {color_name} cloth to become invisible!",
                        (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        except cv2.error as e:
            print("Error writing text on frame:", e)
            continue
 
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = None
        for lower, upper in masks:
            temp_mask = cv2.inRange(hsv, lower, upper)
            mask = temp_mask if mask is None else mask + temp_mask
 
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=2)
        mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=1)
        inverse_mask = cv2.bitwise_not(mask)
 
        res1 = cv2.bitwise_and(background, background, mask=mask)
        res2 = cv2.bitwise_and(frame, frame, mask=inverse_mask)
        output_frame = cv2.addWeighted(res1, 1, res2, 1, 0)
 
        if record:
            out.write(output_frame)
 
        cv2.imshow("Invisibility Cloak", output_frame)
 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
 
    cap.release()
    if record:
        out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_cloak(color_name='iceblue', record=True)


Capturing background... Please step out of the frame.


2025-04-22 11:19:22.700 python[7988:9503561] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-22 11:19:22.700 python[7988:9503561] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Background captured.
Starting invisibility effect... Press 'q' to quit.


: 